In [2]:
import os
import pandas as pd
import openpyxl

# Define the directory where your CSV files are located
csv_directory = 'csvs'  # Change this to the directory containing your CSV files

# Define the directory where you want to save the filtered CSV files
output_csv_directory = 'output_csv'  # Change this to the directory where you want to save the filtered CSV files

# Create the output directory for CSV files if it doesn't exist
os.makedirs(output_csv_directory, exist_ok=True)

# Define the directory where you want to save the Excel file
output_excel_directory = 'output_excel'  # Change this to the directory where you want to save the Excel file

# Create the output directory for the Excel file if it doesn't exist
os.makedirs(output_excel_directory, exist_ok=True)

# Specify the Excel file path
excel_file_path = 'GENeSYS-MOD_User_Input_Settings_v01_phe_05-09-2023.xlsx'  # Replace with the path to your Excel file

# Read the Excel file into a Pandas DataFrame
df = pd.read_excel(excel_file_path, engine='openpyxl')


# Filter the DataFrame to keep rows where the fifth column has a value of 1
filtered_df = df[df.iloc[:, 4] == 1]  # Assuming the fifth column is indexed at 4 (0-based index)


# Create a list of unique values in the fourth column of the filtered DataFrame
unique_values = filtered_df.iloc[:, 3].unique()  # Assuming the fourth column is indexed at 3 (0-based index)

print(unique_values)

['DE-BY-UF' 'NO11' 'NO1' 'DE-BY' 'CN-Cen' 'CN-Cen-Henan' 'AT' 'BE' 'BG'
 'CH' 'CZ' 'DE' 'DK' 'EE' 'FI' 'FR' 'GR' 'HR' 'IE' 'IT' 'LT' 'LU' 'LV'
 'NL' 'NO' 'PL' 'PT' 'RO' 'SE' 'SI' 'SK' 'TR' 'UK' 'NONEU_Balkan']


In [3]:
# Initialize an empty dictionary to store DataFrames
data_frames = {}

In [7]:
# Initialize the Excel writer
output_excel_file_path = os.path.join(output_excel_directory, 'output.xlsx')
with pd.ExcelWriter(output_excel_file_path, engine='openpyxl') as writer:
    for filename in os.listdir(csv_directory):
        
        if filename.endswith('.csv'):
            # Construct the full path to the CSV file
            csv_file_path = os.path.join(csv_directory, filename)
            
            # Get the worksheet name without the .csv extension
            worksheet_name = os.path.splitext(filename)[0]
            
            # Construct the full path to the output CSV file
            output_csv_file_path = os.path.join(output_csv_directory, filename)
            
            if worksheet_name == 'Sets': 
                df = pd.read_csv(csv_file_path, delimiter=';')

                # Store the DataFrame in the dictionary with the filename (without extension) as the key
                data_frames[worksheet_name] = df
                
                # Filter region
                if "Region" in df.columns:
                    # Filter the DataFrame based on whether the values in the "Region" column are present in unique_values
                    filtered_df = df[df["Region"].isin(unique_values)]
                else:
                    filtered_df = df.copy()

                # Write the DataFrame to an Excel worksheet with the same filename (without extension)
                filtered_df.to_excel(writer, sheet_name=worksheet_name, index=False, header=True)

                # Save the filtered DataFrame to the CSV file without the index
                filtered_df.to_csv(output_csv_file_path, index=False, header=True, decimal=',')
            
            else:
            
                df_first = pd.read_csv(csv_file_path, delimiter=';',header=None)
                df_header = df_first.iloc[:4] #shweta
                df_header = df_header.apply(lambda x: x.apply(lambda y: str(y).replace('.0', '')))
                df_header = df_header.iloc[:4, :1]

                # Read the CSV file into a Pandas DataFrame, skipping the first 4 rows
                df = pd.read_csv(csv_file_path, delimiter=';', skiprows=4)

                if "Region" in df.columns:
                    # Filter the DataFrame based on whether the values in the "Region" column are present in unique_values
                    filtered_df = df[df["Region"].isin(unique_values)]
                    if "Region2" in df.columns:
                         filtered_df = filtered_df[filtered_df["Region2"].isin(unique_values)]  
                else:
                    # If "Region" column is not found, copy the entire DataFrame
                    filtered_df = df.copy()

                filtered_df = filtered_df.columns.to_frame().T.append(filtered_df, ignore_index=True)
                filtered_df.columns = range(len(df.columns))

                df_final = pd.concat([df_header, filtered_df], axis=0)

                df_final.replace('nan', '', inplace=True)
                df_final.apply(lambda x: x.apply(lambda y: str(y).replace('.', ',')))

                # Store the DataFrame in the dictionary with the filename (without extension) as the key
                data_frames[worksheet_name] = df_final

                # Write the DataFrame to an Excel worksheet with the same filename (without extension)
                df_final.to_excel(writer, sheet_name=worksheet_name, index=False, header=False)


                # Save the filtered DataFrame to the CSV file without the index
                df_final.to_csv(output_csv_file_path, index=False, header=False, decimal=',')

/var/folders/ww/34wy9_qd5f5_r6d15cv5m8rh0000gn/T/ipykernel_80540/336641116.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  filtered_df = filtered_df.columns.to_frame().T.append(filtered_df, ignore_index=True)
/var/folders/ww/34wy9_qd5f5_r6d15cv5m8rh0000gn/T/ipykernel_80540/336641116.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  filtered_df = filtered_df.columns.to_frame().T.append(filtered_df, ignore_index=True)
/var/folders/ww/34wy9_qd5f5_r6d15cv5m8rh0000gn/T/ipykernel_80540/336641116.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  filtered_df = filtered_df.columns.to_frame().T.append(filtered_df, ignore_index=True)
/var/folders/ww/34wy9_qd5f5_r6d15cv5m8rh0000gn/T/ipykernel_80540/336641116.py:54: FutureWar

/var/folders/ww/34wy9_qd5f5_r6d15cv5m8rh0000gn/T/ipykernel_80540/336641116.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  filtered_df = filtered_df.columns.to_frame().T.append(filtered_df, ignore_index=True)
/var/folders/ww/34wy9_qd5f5_r6d15cv5m8rh0000gn/T/ipykernel_80540/336641116.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  filtered_df = filtered_df.columns.to_frame().T.append(filtered_df, ignore_index=True)
/var/folders/ww/34wy9_qd5f5_r6d15cv5m8rh0000gn/T/ipykernel_80540/336641116.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  filtered_df = filtered_df.columns.to_frame().T.append(filtered_df, ignore_index=True)
/var/folders/ww/34wy9_qd5f5_r6d15cv5m8rh0000gn/T/ipykernel_80540/336641116.py:54: FutureWar